# Import library

In [2]:
import pandas as pd
import numpy as np

# Clean dataset

## Load data

In [3]:
df_product = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/product_raw.csv")
df_product = df_product[df_product['ACTIVE'] == 'Y']
df_product.drop(index = df_product[df_product['REVIEW_NO'] <= 5].index, inplace=True)
df_product.rename(columns={'SECTION_L1': 'CID', 'TYPE_ID': 'SCID'}, inplace=True)

df_cat = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/category.csv")
df_brand = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/brand_name.csv")

df_user = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/user.csv")
df_user.rename(columns={'ID': 'UID'}, inplace=True)

df_review = pd.read_csv("/home/music/Desktop/measure_model/db_cosmenet/review.csv", low_memory=False)
df_review = df_review[df_review['ACTION'] == 1]

In [4]:
df_review = df_review[df_review['EID'].isin(df_product['EID']) & df_review['UID'].isin(df_user['UID'])].reset_index(drop=True)
df_product = df_product[df_product['EID'].isin(df_review['EID'])].reset_index(drop=True)
df_user = df_user[df_user['UID'].isin(df_review['UID'])].reset_index(drop=True)

## Product

In [11]:
# clean category field
df_product_clean = df_product.copy()
row_scid_is_cid = df_product_clean['SCID'].isin(df_cat[df_cat['DEPTH_LEVEL'] == 1]['ID'].values)
df_product_clean.loc[row_scid_is_cid, 'CID'] = df_product_clean.loc[row_scid_is_cid, 'SCID']

row_scid_is_mid = df_product_clean['SCID'].isin(df_cat[df_cat['DEPTH_LEVEL'] == 2]['ID'].values)
df_product_clean.loc[row_scid_is_mid, 'CID'] = df_product_clean.loc[row_scid_is_mid, 'SECTION_L2']

row_scid_notin_scid = ~(df_product_clean['SCID'].isin(df_cat[df_cat['DEPTH_LEVEL'] == 3]['ID'].values))

df_product_clean['SCID'] = df_product_clean['SCID'].astype(object)
df_product_clean['CID'] = df_product_clean['CID'].astype(object)
df_product_clean['BID'] = df_product_clean['BID'].astype(object)

df_product_clean.loc[~df_product_clean['SCID'].isna(), 'SCID'] = df_product_clean[~df_product_clean['SCID'].isna()]['SCID'].astype(int)
df_product_clean.loc[~df_product_clean['CID'].isna(), 'CID'] = df_product_clean[~df_product_clean['CID'].isna()]['CID'].astype(int)
df_product_clean.loc[~df_product_clean['BID'].isna(), 'BID'] = df_product_clean[~df_product_clean['BID'].isna()]['BID'].astype(int)

In [12]:
# create RATE_TEXT
for lw_bound in np.arange(0, 5, 0.5):
    up_bound = lw_bound + 0.5
    df_product_clean.loc[(df_product_clean['RATE'] >= lw_bound) & (df_product_clean['RATE'] < up_bound), 'RATE_TEXT'] = str(lw_bound) + ' - ' + str(up_bound)
df_product_clean = df_product_clean[['EID', 'BID', 'SCID', 'CID', 'RATE_TEXT']]
df_product_clean.reset_index(drop=True, inplace=True)
df_product_clean.head(1)

,EID,BID,SCID,CID,RATE_TEXT
0,13001,1528,248,62,4.5 - 5.0


In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df_product_clean[['SCID', 'CID', 'RATE_TEXT']])
arr_ohe = enc.transform(df_product_clean[['SCID', 'CID', 'RATE_TEXT']]).toarray()
feature_product = enc.get_feature_names_out(['SCID', 'CID', 'RATE_TEXT'])
df_product_clean = df_product_clean.join(pd.DataFrame(arr_ohe, columns=feature_product))
df_product_clean.drop(columns=['SCID', 'CID', 'RATE_TEXT'], inplace=True)
df_product_clean.head(1)

,EID,BID,SCID_56,SCID_68,SCID_69,SCID_73,SCID_77,SCID_78,SCID_79,SCID_82,...,CID_nan,RATE_TEXT_1.0 - 1.5,RATE_TEXT_1.5 - 2.0,RATE_TEXT_2.0 - 2.5,RATE_TEXT_2.5 - 3.0,RATE_TEXT_3.0 - 3.5,RATE_TEXT_3.5 - 4.0,RATE_TEXT_4.0 - 4.5,RATE_TEXT_4.5 - 5.0,RATE_TEXT_nan
0,13001,1528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Review

In [15]:
col_pos = ['UID', 'EID', 'BID', 'DATE_CREATE', 'POINT']

dataset = df_review.sort_values(['UID', 'DATE_CREATE']).reset_index(drop=True)[col_pos]
dataset['one'] = 1
dataset['sample_num'] = dataset.groupby('UID')['one'].cumsum()
dataset['target'] = dataset.groupby('UID')['EID'].shift(-1)
dataset['mean_rate'] = dataset.groupby('UID')['POINT'].cumsum() / dataset['sample_num']

dataset['prev_movies'] = dataset['EID'].apply(lambda x: str(x)).astype(str).reset_index(drop=True)
dataset['prev_movies'] = dataset.groupby('UID')['prev_movies'].apply(lambda x: (x + ' ').cumsum().str.strip()).reset_index(drop=True)
dataset['prev_movies'] = dataset['prev_movies'].apply(lambda x: x.split())
dataset.head()

,UID,EID,BID,DATE_CREATE,POINT,one,sample_num,target,mean_rate,prev_movies
0,4,17566,2663,2015-06-25 11:38:37.000,4,1,1,17704.0,4.0,[17566]
1,4,17704,2620,2015-06-26 09:45:18.000,4,1,2,293.0,4.0,"[17566, 17704]"
2,4,293,2584,2015-06-29 09:53:34.000,4,1,3,15031.0,4.0,"[17566, 17704, 293]"
3,4,15031,2597,2015-06-30 19:11:22.000,4,1,4,9857.0,4.0,"[17566, 17704, 293, 15031]"
4,4,9857,245,2015-07-02 11:20:13.000,4,1,5,19005.0,4.0,"[17566, 17704, 293, 15031, 9857]"


In [20]:
dataset = dataset.merge(df_product_clean[['EID'] + feature_product.tolist()], on='EID', how='left')
for genre in feature_product:
    dataset = dataset.join(pd.DataFrame((dataset[genre]*dataset['POINT']).values, columns=[f'{genre}_rate']))
    dataset[genre] = dataset.groupby('UID')[genre].cumsum()
    dataset[f'{genre}_rate'] = dataset.groupby('UID')[f'{genre}_rate'].cumsum() / dataset[genre]
    
dataset[feature_product] = dataset[feature_product].apply(lambda x: x / dataset['sample_num'])
dataset.head()

,UID,EID,BID,DATE_CREATE,POINT,one,sample_num,target,mean_rate,prev_movies,...,CID_nan_rate,RATE_TEXT_1.0 - 1.5_rate,RATE_TEXT_1.5 - 2.0_rate,RATE_TEXT_2.0 - 2.5_rate,RATE_TEXT_2.5 - 3.0_rate,RATE_TEXT_3.0 - 3.5_rate,RATE_TEXT_3.5 - 4.0_rate,RATE_TEXT_4.0 - 4.5_rate,RATE_TEXT_4.5 - 5.0_rate,RATE_TEXT_nan_rate
0,4,17566,2663,2015-06-25 11:38:37.000,4,1,1,17704.0,4.0,[17566],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,4,17704,2620,2015-06-26 09:45:18.000,4,1,2,293.0,4.0,"[17566, 17704]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,4,293,2584,2015-06-29 09:53:34.000,4,1,3,15031.0,4.0,"[17566, 17704, 293]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,4,15031,2597,2015-06-30 19:11:22.000,4,1,4,9857.0,4.0,"[17566, 17704, 293, 15031]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,4,9857,245,2015-07-02 11:20:13.000,4,1,5,19005.0,4.0,"[17566, 17704, 293, 15031, 9857]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
